# Run MMBT Experiments

This notebook shows the end-to-end pipeline to fine-tune pre-trained MMBT model for multimodal (text and image) classification on our dataset.

Parts of this pipeline are adapted from the
Huggingface `run_mmimdb.py` script to execute the MMBT model. This code can
be accessed [here.](https://github.com/huggingface/transformers/blob/8ea412a86faa8e9edeeb6b5c46b08def06aa03ea/examples/research_projects/mm-imdb/run_mmimdb.py#L305)

## Skip unless on Google Colab

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/LAP_MMBT
%pwd

/content/drive/MyDrive/LAP_MMBT


'/content/drive/MyDrive/LAP_MMBT'

Before running the cell below, make sure to select 'GPU' runtime type

In [3]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


## Install Huggingface Library

These should have been installed during your environment set-up; you only need to run these cells in Google Colab.

In [4]:
!pip install transformers

     |████████████████████████████████| 1.8MB 9.0MB/s 
     |████████████████████████████████| 890kB 46.2MB/s 
     |████████████████████████████████| 3.2MB 50.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=9af3a36ceee77394672eb57f65c79e678b85f867bed61d8f68dbba52f6535b50
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


# Data directories and file paths


In [5]:
train_file = "image_labels_impression_frontal_train.jsonl" 
val_file = "image_labels_impression_frontal_val.jsonl" 
test_file = "image_labels_impression_frontal_test.jsonl" 

## Import Required Modules

In [6]:
from textBert_utils import set_seed
from MMBT.image import ImageEncoderDenseNet
from MMBT.mmbt_config import MMBTConfig
from MMBT.mmbt import MMBTForClassification

In [7]:
from MMBT.mmbt_utils import JsonlDataset, get_image_transforms, get_labels, load_examples, collate_fn

In [8]:
import argparse

In [9]:
import glob
import logging
import random
import json
import os
from collections import Counter
import numpy as np
from matplotlib.pyplot import imshow

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [10]:
from sklearn.metrics import accuracy_score
from tqdm import tqdm, trange

from transformers import (
    WEIGHTS_NAME,
    AdamW,
    AutoConfig,
    AutoModel,
    AutoTokenizer,
    get_linear_schedule_with_warmup,
)

try:
    from torch.utils.tensorboard import SummaryWriter
except ImportError:
    from tensorboardX import SummaryWriter

In [11]:
from transformers import (
    WEIGHTS_NAME,
    AdamW,
    AutoConfig,
    AutoModel,
    AutoTokenizer,
    get_linear_schedule_with_warmup,
)

# Set-up Experiment Hyperparameters and Arguments

Specify the training, validation, and test files to run the experiment on. The default here is running the model on 'impression' texts.  

To re-make the training, validation, and test data, please refer to the information in the **data/** directory.  

Change the default values in the parser.add_argument function for the hyperparameters that you want to specify in the following cell or use the default option.  

For multiple experiment runs, please make sure to change the `output_dir` argument so that new results don't overwrit existing ones.

The arguments specified here are the same as in the `run_mmimdb.py` file 
in the [Huggingface example implementation of MMBT.](https://github.com/huggingface/transformers/blob/8ea412a86faa8e9edeeb6b5c46b08def06aa03ea/examples/research_projects/mm-imdb/run_mmimdb.py#L305)

In [12]:
parser = argparse.ArgumentParser(f'Project Hyperparameters and Other Configurations Argument Parser')

parser = argparse.ArgumentParser()

# Required parameters
parser.add_argument(
    "--data_dir",
    default="data/json",
    type=str,
    help="The input data dir. Should contain the .jsonl files.",
)
parser.add_argument(
    "--model_name",
    default="bert-base-uncased",
    type=str,
    help="model identifier from huggingface.co/models",
)
parser.add_argument(
    "--output_dir",
    default="mmbt_output_impression",
    type=str,
    help="The output directory where the model predictions and checkpoints will be written.",
)

    
parser.add_argument(
    "--config_name", default="bert-base-uncased", type=str, help="Pretrained config name if not the same as model_name"
)
parser.add_argument(
    "--tokenizer_name",
    default="bert-base-uncased",
    type=str,
    help="Pretrained tokenizer name or path if not the same as model_name",
)

parser.add_argument("--train_batch_size", default=32, type=int, help="Batch size for training.")
parser.add_argument(
    "--eval_batch_size", default=32, type=int, help="Batch size for evaluation."
)
parser.add_argument(
    "--max_seq_length",
    default=256,
    type=int,
    help="The maximum total input sequence length after tokenization. Sequences longer "
    "than this will be truncated, sequences shorter will be padded.",
)
parser.add_argument(
    "--num_image_embeds", default=3, type=int, help="Number of Image Embeddings from the Image Encoder"
)
parser.add_argument("--do_train", default=True, type=bool, help="Whether to run training.")
parser.add_argument("--do_eval", default=True, type=bool, help="Whether to run eval on the dev set.")
parser.add_argument(
    "--evaluate_during_training", default=True, type=bool, help="Rul evaluation during training at each logging step."
)


parser.add_argument(
    "--gradient_accumulation_steps",
    type=int,
    default=1,
    help="Number of updates steps to accumulate before performing a backward/update pass.",
)
parser.add_argument("--learning_rate", default=5e-5, type=float, help="The initial learning rate for Adam.")
parser.add_argument("--weight_decay", default=0.1, type=float, help="Weight deay if we apply some.")
parser.add_argument("--adam_epsilon", default=1e-8, type=float, help="Epsilon for Adam optimizer.")
parser.add_argument("--max_grad_norm", default=1.0, type=float, help="Max gradient norm.")
parser.add_argument(
    "--num_train_epochs", default=4.0, type=float, help="Total number of training epochs to perform."
)
parser.add_argument("--patience", default=5, type=int, help="Patience for Early Stopping.")
parser.add_argument(
    "--max_steps",
    default=-1,
    type=int,
    help="If > 0: set total number of training steps to perform. Override num_train_epochs.",
)
parser.add_argument("--warmup_steps", default=0, type=int, help="Linear warmup over warmup_steps.")

parser.add_argument("--logging_steps", type=int, default=25, help="Log every X updates steps.")
parser.add_argument("--save_steps", type=int, default=25, help="Save checkpoint every X updates steps.")
parser.add_argument(
    "--eval_all_checkpoints",
    default=True, type=bool,
    help="Evaluate all checkpoints starting with the same prefix as model_name ending and ending with step number",
)

parser.add_argument("--num_workers", type=int, default=8, help="number of worker threads for dataloading")

parser.add_argument("--seed", type=int, default=42, help="random seed for initialization")


args = parser.parse_args("")

# Setup CUDA, GPU & distributed training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
args.n_gpu = torch.cuda.device_count() if torch.cuda.is_available() else 0
args.device = device

In [13]:
# Setup Train/Val/Test filenames
args.train_file = train_file
args.val_file = val_file
args.test_file = test_file

## Showing a sample from JsonDataset
i.e. calling "\_\_getitem\_\_"

Note:   
image_end_token is the BERT token id for [SEP].   
image_start_token is the BERT token id for [CLS]. 


In [14]:
tokenizer = AutoTokenizer.from_pretrained(
        args.tokenizer_name if args.tokenizer_name else args.model_name,
        do_lower_case=True,
        cache_dir=None,
    )
train_dataset = load_examples(tokenizer, args)

In [15]:
train_dataset[0]

{'image': tensor([[[ 0.4679,  0.4166,  0.3481,  ..., -0.7822, -0.7822, -0.7993],
          [ 0.4851,  0.5022,  0.5364,  ..., -0.5938, -0.5938, -0.6452],
          [ 0.4166,  0.3823,  0.4337,  ..., -0.4226, -0.4226, -0.4568],
          ...,
          [ 1.8722,  1.9064,  1.9235,  ...,  0.7248,  0.6734,  0.6221],
          [ 1.8208,  1.8550,  1.9064,  ...,  0.7248,  0.6734,  0.6392],
          [ 1.7865,  1.8208,  1.8379,  ...,  0.7077,  0.6906,  0.6563]],
 
         [[ 0.6078,  0.5553,  0.4853,  ..., -0.6702, -0.6702, -0.6877],
          [ 0.6254,  0.6429,  0.6779,  ..., -0.4776, -0.4776, -0.5301],
          [ 0.5553,  0.5203,  0.5728,  ..., -0.3025, -0.3025, -0.3375],
          ...,
          [ 2.0434,  2.0784,  2.0959,  ...,  0.8704,  0.8179,  0.7654],
          [ 1.9909,  2.0259,  2.0784,  ...,  0.8704,  0.8179,  0.7829],
          [ 1.9559,  1.9909,  2.0084,  ...,  0.8529,  0.8354,  0.8004]],
 
         [[ 0.8274,  0.7751,  0.7054,  ..., -0.4450, -0.4450, -0.4624],
          [ 0.8448,


### Training and Evaluating Functions.

In [16]:
def train(args, train_dataset, model, tokenizer):
    """ Train the model """
    
    tb_writer = SummaryWriter()

    train_sampler = RandomSampler(train_dataset)
    train_dataloader = DataLoader(
        train_dataset,
        sampler=train_sampler,
        batch_size=args.train_batch_size,
        collate_fn=collate_fn
    )

    t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args.weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]

    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
    )
    

    # Train!
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args.num_train_epochs)
    logger.info(
        "  Total train batch size = %d",
        args.train_batch_size
        * args.gradient_accumulation_steps)
    logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 0
    tr_loss, logging_loss = 0.0, 0.0
    best_accuracy, n_no_improve = 0, 0
    model.train()
    model.zero_grad()
    optimizer.zero_grad()
    train_iterator = trange(int(args.num_train_epochs), desc="Epoch")
    set_seed(args)  # Added here for reproductibility
    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Training Batch Iteration")
        for step, batch in enumerate(epoch_iterator):
            # model.train()
            # each sample in batch is a tuple
            # batch is the return of the collate_fn function
            # see function definition for data tuple order
            batch = tuple(t.to(args.device) for t in batch)
            labels = batch[5]
            input_ids = batch[0]
            input_modal = batch[2]
            attention_mask = batch[1]
            modal_start_tokens = batch[3]
            modal_end_tokens = batch[4]

            #inputs = {
            #    "input_ids": batch[0],
            #    "input_modal": batch[2],
            #    "attention_mask": batch[1],
            #    "modal_start_tokens": batch[3],
            #    "modal_end_tokens": batch[4],
            #    "labels": batch[5]
            #}

            outputs = model(
                input_modal,
                input_ids=input_ids,
                modal_start_tokens=modal_start_tokens,
                modal_end_tokens=modal_end_tokens,
                attention_mask=attention_mask,
                token_type_ids=None,
                modal_token_type_ids=None,
                position_ids=None,
                modal_position_ids=None,
                head_mask=None,
                inputs_embeds=None,
                labels=labels,
                return_dict=True
            )
            #logits = outputs[0]  # model outputs are always tuple in transformers (see doc)
            #loss = criterion(logits, labels)
            loss = outputs.loss
            logits = outputs.logits
            
            if args.n_gpu > 1:
                loss = loss.mean()  # mean() to average on multi-gpu parallel training
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps


            loss.backward()

            tr_loss += loss.item()
            if (step + 1) % args.gradient_accumulation_steps == 0:

                torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)

                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1

                if args.logging_steps > 0 and global_step % args.logging_steps == 0:
                    logs = {}
                    if args.evaluate_during_training:  
                        # Only evaluate when single GPU otherwise metrics may not average well
                        results = evaluate(args, model, tokenizer)
                        for key, value in results.items():
                            eval_key = "eval_{}".format(key)
                            logs[eval_key] = value

                    loss_scalar = (tr_loss - logging_loss) / args.logging_steps
                    learning_rate_scalar = scheduler.get_last_lr()[0]
                    logs["learning_rate"] = learning_rate_scalar
                    logs["loss"] = loss_scalar
                    logging_loss = tr_loss

                    for key, value in logs.items():
                        tb_writer.add_scalar(key, value, global_step)
                    print(json.dumps({**logs, **{"step": global_step}}))

                if args.save_steps > 0 and global_step % args.save_steps == 0:
                    # Save model checkpoint
                    output_dir = os.path.join(args.output_dir, "checkpoint-{}".format(global_step))
                    if not os.path.exists(output_dir):
                        os.makedirs(output_dir)
                    model_to_save = (
                        model.module if hasattr(model, "module") else model
                    )  # Take care of distributed/parallel training
                    torch.save(model_to_save.state_dict(), os.path.join(output_dir, WEIGHTS_NAME))
                    # uncomment below to be able to save args
                    # torch.save(args, os.path.join(output_dir, "training_args.bin"))
                    logger.info("Saving model checkpoint to %s", output_dir)


        results = evaluate(args, model, tokenizer)
        if results["accuracy"] > best_accuracy:
            best_accuracy = results["accuracy"]
            n_no_improve = 0
        else:
            n_no_improve += 1

        if n_no_improve > args.patience:
            train_iterator.close()
            break

    tb_writer.close()

    return global_step, tr_loss / global_step

In [17]:
def evaluate(args, model, tokenizer, evaluate=True, test=False, prefix=""):
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    eval_output_dir = args.output_dir
    eval_dataset = load_examples(tokenizer, args, evaluate=evaluate, test=test)

    if not os.path.exists(eval_output_dir):
        os.makedirs(eval_output_dir)

    # Note that DistributedSampler samples randomly
    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(
        eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size, collate_fn=collate_fn
    )

    # Eval!
    logger.info("***** Running evaluation {} *****".format(prefix))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args.eval_batch_size)
    eval_loss = 0.0
    nb_eval_steps = 0
    preds = []
    out_label_ids = []
    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        model.eval()
        batch = tuple(t.to(args.device) for t in batch)

        with torch.no_grad():
            batch = tuple(t.to(args.device) for t in batch)
            labels = batch[5]
            input_ids = batch[0]
            input_modal = batch[2]
            attention_mask = batch[1]
            modal_start_tokens = batch[3]
            modal_end_tokens = batch[4]
            
            outputs = model(
                input_modal,
                input_ids=input_ids,
                modal_start_tokens=modal_start_tokens,
                modal_end_tokens=modal_end_tokens,
                attention_mask=attention_mask,
                token_type_ids=None,
                modal_token_type_ids=None,
                position_ids=None,
                modal_position_ids=None,
                head_mask=None,
                inputs_embeds=None,
                labels=labels,
                return_dict=True
            )
            #logits = outputs[0]  # model outputs are always tuple in transformers (see doc)
            #tmp_eval_loss = criterion(logits, labels)
            tmp_eval_loss = outputs.loss
            logits = outputs.logits
            eval_loss += tmp_eval_loss.mean().item()
        nb_eval_steps += 1
        # Move logits and labels to CPU            
        pred = torch.nn.functional.softmax(logits, dim=1).argmax(dim=1).cpu().detach().numpy()
        out_label_id = labels.detach().cpu().numpy()
        preds.append(pred)
        out_label_ids.append(out_label_id)

    eval_loss = eval_loss / nb_eval_steps

    preds = [l for sl in preds for l in sl]
    out_label_ids = [l for sl in out_label_ids for l in sl]

    result = {
        "loss": eval_loss,
        "accuracy": accuracy_score(out_label_ids, preds)
    }

    output_eval_file = os.path.join(eval_output_dir, prefix, "eval_results.txt")
    with open(output_eval_file, "w") as writer:
        logger.info("***** Eval results {} *****".format(prefix))
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
            writer.write("%s = %s\n" % (key, str(result[key])))

    return result


## Training MMBT Model 

Set up logging and the MMBT Model. Similar to the text-only model, check points 
are saved during a similar customizable interval.



In [18]:
# Setup logging
logger = logging.getLogger(__name__)
if not os.path.exists(args.output_dir):
    os.makedirs(args.output_dir)
logging.basicConfig(format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
                    datefmt="%m/%d/%Y %H:%M:%S",
                    filename=os.path.join(args.output_dir, f"{os.path.splitext(args.train_file)[0]}_logging.txt"),
                    level=logging.INFO)
logger.warning("device: %s, n_gpu: %s",
        args.device,
        args.n_gpu
)
# Set the verbosity to info of the Transformers logger (on main process only):

# Set seed
set_seed(args)

In [19]:
# Setup model
labels = get_labels()
num_labels = len(labels)
transformer_config = AutoConfig.from_pretrained(args.config_name if args.config_name else args.model_name)
tokenizer = AutoTokenizer.from_pretrained(
        args.tokenizer_name if args.tokenizer_name else args.model_name,
        do_lower_case=True,
        cache_dir=None,
    )
transformer = AutoModel.from_pretrained(args.model_name, config=transformer_config, cache_dir=None)
img_encoder = ImageEncoderDenseNet(num_image_embeds=args.num_image_embeds)
multimodal_config = MMBTConfig(transformer, img_encoder, num_labels=num_labels, modal_hidden_size=1024)
model = MMBTForClassification(transformer_config, multimodal_config)

model.to(args.device)

logger.info(f"Training/evaluation parameters: {args}")

# Training
if args.do_train:
    train_dataset = load_examples(tokenizer, args)
    # criterion = nn.CrossEntropyLoss
    global_step, tr_loss = train(args, train_dataset, model, tokenizer)
    logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()
    logger.info("Saving model checkpoint to %s", args.output_dir)
    # Save a trained model, configuration and tokenizer using `save_pretrained()`.
    # They can then be reloaded using `from_pretrained()`
    model_to_save = (model.module if hasattr(model, "module") else model)  # Take care of distributed/parallel training
    torch.save(model_to_save.state_dict(), os.path.join(args.output_dir, WEIGHTS_NAME))
    tokenizer.save_pretrained(args.output_dir)
    transformer_config.save_pretrained(args.output_dir)
    # Good practice: save your training arguments together with the trained model
    torch.save(args, os.path.join(args.output_dir, "training_args.bin"))

    # Load a trained model and vocabulary that you have fine-tuned
    model = MMBTForClassification(transformer_config, multimodal_config)
    model.load_state_dict(torch.load(os.path.join(args.output_dir, WEIGHTS_NAME)))
    tokenizer = AutoTokenizer.from_pretrained(args.output_dir)
    model.to(args.device)
logger.info("***** Training Finished *****")

Training Batch Iteration:  39%|███▉      | 24/61 [05:31<08:08, 13.19s/it]

Evaluating:   0%|          | 0/21 [00:00<?, ?it/s]

Evaluating:   5%|▍         | 1/21 [00:12<04:04, 12.24s/it]

Evaluating:  10%|▉         | 2/21 [00:25<03:57, 12.50s/it]

Evaluating:  14%|█▍        | 3/21 [00:37<03:44, 12.46s/it]

Evaluating:  19%|█▉        | 4/21 [00:50<03:35, 12.70s/it]

Evaluating:  24%|██▍       | 5/21 [01:03<03:22, 12.64s/it]

Evaluating:  29%|██▊       | 6/21 [01:16<03:09, 12.61s/it]

Evaluating:  33%|███▎      | 7/21 [01:28<02:54, 12.49s/it]

Evaluating:  38%|███▊      | 8/21 [01:41<02:44, 12.68s/it]

Evaluating:  43%|████▎     | 9/21 [01:53<02:30, 12.58s/it]

Evaluating:  48%|████▊     | 10/21 [02:07<02:21, 12.85s/it]

Evaluating:  52%|█████▏    | 11/21 [02:19<02:06, 12.69s/it]

Evaluating:  57%|█████▋    | 12/21 [02:31<01:53, 12.56s/it]

Evaluating:  62%|██████▏   | 13/21 [02:44<01:39, 12.49s/it]

Evaluating:  67%|██████▋   | 14/21 [02:56<01:27, 12.49s/it]

Evaluating:  71%|███████▏  |

{"eval_loss": 0.3071372416757402, "eval_accuracy": 0.8952234206471494, "learning_rate": 4.487704918032787e-05, "loss": 0.46950038194656374, "step": 25}



Training Batch Iteration:  80%|████████  | 49/61 [15:06<02:33, 12.78s/it]

Evaluating:   0%|          | 0/21 [00:00<?, ?it/s]

Evaluating:   5%|▍         | 1/21 [00:00<00:10,  1.99it/s]

Evaluating:  10%|▉         | 2/21 [00:01<00:09,  1.94it/s]

Evaluating:  14%|█▍        | 3/21 [00:01<00:09,  1.97it/s]

Evaluating:  19%|█▉        | 4/21 [00:02<00:08,  2.00it/s]

Evaluating:  24%|██▍       | 5/21 [00:02<00:08,  1.98it/s]

Evaluating:  29%|██▊       | 6/21 [00:03<00:08,  1.83it/s]

Evaluating:  33%|███▎      | 7/21 [00:03<00:07,  1.88it/s]

Evaluating:  38%|███▊      | 8/21 [00:04<00:07,  1.82it/s]

Evaluating:  43%|████▎     | 9/21 [00:04<00:06,  1.81it/s]

Evaluating:  48%|████▊     | 10/21 [00:05<00:06,  1.83it/s]

Evaluating:  52%|█████▏    | 11/21 [00:05<00:05,  1.82it/s]

Evaluating:  57%|█████▋    | 12/21 [00:06<00:04,  1.82it/s]

Evaluating:  62%|██████▏   | 13/21 [00:06<00:04,  1.91it/s]

Evaluating:  67%|██████▋   | 14/21 [00:07<00:03,  1.89it/s]

Evaluating:  71%|███████▏  

{"eval_loss": 0.1675085269269489, "eval_accuracy": 0.9414483821263482, "learning_rate": 3.975409836065574e-05, "loss": 0.20768402725458146, "step": 50}



Training Batch Iteration: 100%|██████████| 61/61 [17:50<00:00, 17.54s/it]

Training Batch Iteration:  21%|██▏       | 13/61 [00:13<00:50,  1.05s/it]

Evaluating:   0%|          | 0/21 [00:00<?, ?it/s]

Evaluating:   5%|▍         | 1/21 [00:00<00:10,  1.98it/s]

Evaluating:  10%|▉         | 2/21 [00:01<00:09,  1.92it/s]

Evaluating:  14%|█▍        | 3/21 [00:01<00:09,  1.96it/s]

Evaluating:  19%|█▉        | 4/21 [00:02<00:08,  1.98it/s]

Evaluating:  24%|██▍       | 5/21 [00:02<00:08,  1.98it/s]

Evaluating:  29%|██▊       | 6/21 [00:03<00:08,  1.82it/s]

Evaluating:  33%|███▎      | 7/21 [00:03<00:07,  1.89it/s]

Evaluating:  38%|███▊      | 8/21 [00:04<00:07,  1.82it/s]

Evaluating:  43%|████▎     | 9/21 [00:04<00:06,  1.81it/s]

Evaluating:  48%|████▊     | 10/21 [00:05<00:05,  1.86it/s]

Evaluating:  52%|█████▏    | 11/21 [00:05<00:05,  1.84it/s]

Evaluating:  57%|█████▋    | 12/21 [00:06<00:04,  1.83it/s]

Evaluating:  62%|██████▏   | 13/21 [00:06<00:04,  1.91it/s]

Evaluating:  

...


Training Batch Iteration: 100%|██████████| 61/61 [01:51<00:00,  1.82s/it]

Training Batch Iteration:  26%|██▌       | 16/61 [00:18<00:52,  1.17s/it]

Evaluating:   0%|          | 0/21 [00:00<?, ?it/s]

Evaluating:   5%|▍         | 1/21 [00:00<00:10,  1.99it/s]

Evaluating:  10%|▉         | 2/21 [00:01<00:09,  1.93it/s]

Evaluating:  14%|█▍        | 3/21 [00:01<00:09,  1.96it/s]

Evaluating:  19%|█▉        | 4/21 [00:02<00:08,  1.98it/s]

Evaluating:  24%|██▍       | 5/21 [00:02<00:08,  1.96it/s]

Evaluating:  29%|██▊       | 6/21 [00:03<00:08,  1.81it/s]

Evaluating:  33%|███▎      | 7/21 [00:03<00:07,  1.88it/s]

Evaluating:  38%|███▊      | 8/21 [00:04<00:07,  1.81it/s]

Evaluating:  43%|████▎     | 9/21 [00:04<00:06,  1.80it/s]

Evaluating:  48%|████▊     | 10/21 [00:05<00:05,  1.84it/s]

Evaluating:  52%|█████▏    | 11/21 [00:05<00:05,  1.84it/s]

Evaluating:  57%|█████▋    | 12/21 [00:06<00:04,  1.84it/s]

Evaluating:  62%|██████▏   | 13/21 [00:06<00:04,  1.93it/s]

Evaluating:  

{"eval_loss": 0.109706214074755, "eval_accuracy": 0.9722650231124808, "learning_rate": 9.016393442622952e-06, "loss": 0.07073369422927499, "step": 200}



Training Batch Iteration:  67%|██████▋   | 41/61 [01:00<00:21,  1.08s/it]

Evaluating:   0%|          | 0/21 [00:00<?, ?it/s]

Evaluating:  10%|▉         | 2/21 [00:01<00:09,  1.95it/s]

Evaluating:  14%|█▍        | 3/21 [00:01<00:09,  1.98it/s]

Evaluating:  19%|█▉        | 4/21 [00:02<00:08,  2.01it/s]

Evaluating:  24%|██▍       | 5/21 [00:02<00:08,  2.00it/s]

Evaluating:  29%|██▊       | 6/21 [00:03<00:08,  1.84it/s]

Evaluating:  33%|███▎      | 7/21 [00:03<00:07,  1.90it/s]

Evaluating:  38%|███▊      | 8/21 [00:04<00:07,  1.85it/s]

Evaluating:  43%|████▎     | 9/21 [00:04<00:06,  1.83it/s]

Evaluating:  48%|████▊     | 10/21 [00:05<00:05,  1.88it/s]

Evaluating:  52%|█████▏    | 11/21 [00:05<00:05,  1.87it/s]

Evaluating:  57%|█████▋    | 12/21 [00:06<00:04,  1.86it/s]

Evaluating:  62%|██████▏   | 13/21 [00:06<00:04,  1.95it/s]

Evaluating:  67%|██████▋   | 14/21 [00:07<00:03,  1.93it/s]

Evaluating:  71%|███████▏  | 15/21 [00:07<00:03,  1.85it/s]

Evaluating:  76%|███████▌ 

{"eval_loss": 0.12642938399776107, "eval_accuracy": 0.9691833590138675, "learning_rate": 3.89344262295082e-06, "loss": 0.016114526093006135, "step": 225}



Training Batch Iteration: 100%|██████████| 61/61 [01:37<00:00,  1.59s/it]

Epoch: 100%|██████████| 4/4 [23:38<00:00, 354.66s/it]


## Evaluating on the Test Set



In [20]:
# Evaluation
results = {}
if args.do_eval:
    checkpoints = [args.output_dir]
    if args.eval_all_checkpoints:
        checkpoints = list(os.path.dirname(c) 
        for c in sorted(glob.glob(args.output_dir + "/**/" + 
                                  WEIGHTS_NAME, recursive=False)))
        # recursive=False because otherwise the parent diretory gets included
        # which is not what we want; only subdirectories

    logger.info("Evaluate the following checkpoints: %s", checkpoints)

    for checkpoint in checkpoints:
        global_step = checkpoint.split("-")[-1] if len(checkpoints) > 1 else ""
        prefix = checkpoint.split("/")[-1] if checkpoint.find("checkpoint") != -1 else ""
        model = MMBTForClassification(transformer_config, multimodal_config)
        checkpoint = os.path.join(checkpoint, 'pytorch_model.bin')
        model.load_state_dict(torch.load(checkpoint))
        model.to(args.device)
        result = evaluate(args, model, tokenizer, evaluate=True, test=True, prefix=prefix)
        result = dict((k + "_{}".format(global_step), v) for k, v in result.items())
        results.update(result)

results.keys()

Evaluating: 100%|██████████| 21/21 [00:10<00:00,  2.03it/s]


dict_keys(['loss_100', 'accuracy_100', 'loss_125', 'accuracy_125', 'loss_150', 'accuracy_150', 'loss_175', 'accuracy_175', 'loss_200', 'accuracy_200', 'loss_225', 'accuracy_225', 'loss_25', 'accuracy_25', 'loss_50', 'accuracy_50', 'loss_75', 'accuracy_75'])

In [21]:
results

{'accuracy_100': 0.9492307692307692,
 'accuracy_125': 0.963076923076923,
 'accuracy_150': 0.9046153846153846,
 'accuracy_175': 0.9692307692307692,
 'accuracy_200': 0.9646153846153847,
 'accuracy_225': 0.9692307692307692,
 'accuracy_25': 0.8969230769230769,
 'accuracy_50': 0.9461538461538461,
 'accuracy_75': 0.963076923076923,
 'loss_100': 0.13056670280084723,
 'loss_125': 0.15960173424155938,
 'loss_150': 0.4642904294388635,
 'loss_175': 0.1228057337215259,
 'loss_200': 0.1284510674221175,
 'loss_225': 0.1478217844407828,
 'loss_25': 0.2989686181147893,
 'loss_50': 0.15678607938545092,
 'loss_75': 0.16127859109214374}

## Saving Test Eval Results

The code automatically saved evaluation result from each checkpoint in its respective folder. This next cell simply saves all of them in one place.

In [22]:
with open('mmbt_both_front_eval_results.txt', mode='w', encoding='utf-8') as out_f:
    print(results, file=out_f)